In [23]:
import sqlite3
import os
import pickle
import pandas as pd
import sys

execfile('Imports.py')

from Modules.Classes import Simulation

#Allow terminal input argument
# narg = len(sys.argv)
# if __name__ == "__main__" and narg>1:
#     expName = sys.argv[1] #idx 1 arg from terminal is the experiment name
#     compilation = False #by default not a compilation of exps
#     if narg>2:
#         compilation = sys.argv[2] #idx 2 must be Boolean and indicates if experiment db is a compilation of multiple exps
# else:
expName = 'xcrABC' #default experiment name
compilation = False

#Extract only specific conditions. Specify as a list.
extractcond = None

#This list of exps are known to be compilations:
if expName in ['pooled','5con','5con_s']:
    compilation = True

    
#Toggle construction of trialset pickle without first trial?
# This is in addition to the usual.
# Regular full set of trials is always constructed.
no1st = False

if compilation:
    pluralStr = 's'
else:
    pluralStr = ''

if len(expName)==0:
    expNameExt = ''
else:
    expNameExt = '-'
    
con = sqlite3.connect('experiment{}{}{}.db'.format(pluralStr,expNameExt,expName))
participants = pd.read_sql_query("SELECT participant, condition, gentype from participants", con)
generation = pd.read_sql_query("SELECT * from generation", con)
alphas = pd.read_sql_query("SELECT * from alphas", con)
stimuli = pd.read_sql_query("SELECT * from stimuli", con).values
con.close()

print('Extracting trialsets from experiment: {}'.format(expName))

# create categories mapping
mapping = pd.DataFrame(columns = ['condition', 'categories'])
for i in alphas.columns:
	As = alphas[i].values.flatten()
	mapping = mapping.append(
		dict(condition = i, categories =[As]), 
		ignore_index = True
	)
        
# merge categories into generation
generation = pd.merge(generation, participants, on='participant')
generation = pd.merge(generation, mapping, on='condition')



Extracting trialsets from experiment: xcrABC


In [31]:
genbatch = generation[generation.gentype==1]

In [32]:
genbatch

,category,participant,rt,stimulus,trial,condition,gentype,categories
0,Beta,3,4391,179,1,Cluster,1,"[[583, 392, 1083, 1093]]"
1,Beta,3,11435,29,0,Cluster,1,"[[583, 392, 1083, 1093]]"
2,Beta,3,1799,66,3,Cluster,1,"[[583, 392, 1083, 1093]]"
3,Beta,3,3421,169,2,Cluster,1,"[[583, 392, 1083, 1093]]"
4,Beta,3,3931,120,5,Cluster,1,"[[583, 392, 1083, 1093]]"
5,Beta,3,4291,308,4,Cluster,1,"[[583, 392, 1083, 1093]]"
6,Beta,3,5667,218,7,Cluster,1,"[[583, 392, 1083, 1093]]"
7,Beta,3,4741,217,6,Cluster,1,"[[583, 392, 1083, 1093]]"
40,Beta,132,11277,2349,1,Cluster,1,"[[583, 392, 1083, 1093]]"
41,Beta,132,19084,2499,0,Cluster,1,"[[583, 392, 1083, 1093]]"


In [43]:
participants[participants.gentype==2].participant

3       37
7      126
11     130
20     144
24     149
25     150
28     153
35     163
39     167
41     170
45     174
49     178
51     181
53     183
55     185
56     188
60     194
63     198
65     200
68     206
73     211
96     245
98     247
106    258
109    261
114    266
115    267
116    268
118    273
128    285
      ... 
165    337
166    338
169    344
171    347
172    348
173    350
174    351
175    352
181    361
182    363
186    371
188    373
189    374
192    378
197    385
199    388
200    390
201    391
208    398
210    402
212    404
213    405
215    408
217    410
221    420
223    422
225    425
226    426
228    428
234    434
Name: participant, Length: 76, dtype: int64

In [47]:
with open('pickles/xcrC.p','rb') as f:
    trials = pickle.load(f)
print(trials.participants)
print(len(trials.participants))
trials.Set

[ 37. 126. 130. 144. 149. 150. 153. 163. 167. 170. 174. 178. 181. 183.
 185. 188. 194. 198. 200. 206. 211. 245. 247. 258. 261. 266. 267. 268.
 273. 285. 286. 288. 290. 293. 296. 301. 306. 310. 312. 314. 316. 318.
 319. 321. 330. 334. 337. 338. 344. 347. 348. 350. 351. 352. 361. 363.
 371. 373. 374. 378. 385. 388. 390. 391. 398. 402. 404. 405. 408. 410.
 420. 422. 425. 426. 428. 434.]
76


[{'categories': [array([ 403,  818, 1479, 1890]), array([], dtype=float64)],
  'participant': [array([], dtype=int64),
   array([ 37, 130, 144, 149, 150, 181, 183, 194, 247, 258, 285, 286, 288,
          290, 310, 312, 318, 330, 334, 337, 363, 373, 402, 420, 422, 426])],
  'response': [array([], dtype=int64),
   array([1899,  299,  621, 2499, 1202,   49,   18, 1119, 2353, 2336, 1849,
          2499,   49, 2249,   49, 2104,  982, 2349, 1386,   49, 2499,   49,
             0, 2499, 2499, 2450])],
  'wrap_ax': [array([], dtype=int64),
   array([None, None, None, None, None, None, None, None, None, None, None,
          None, None, None, None, None, None, None, None, None, None, None,
          None, None, None, None], dtype=object)]},
 {'categories': [array([ 403,  818, 1479, 1890]), array([1899])],
  'participant': [array([], dtype=int64), array([37])],
  'response': [array([], dtype=int64), array([2498])],
  'wrap_ax': [array([], dtype=int64), array([None], dtype=object)]},
 {'categorie